# Gaze data from LSL

In [ ]:
import cv2
import pyxdf
import numpy as np

# Load XDF file 
xdf_file = 'tobii_test_trial_lsl.xdf'
streams, header = pyxdf.load_xdf(xdf_file)

# Load Tobii gaze coordinates
gaze_stream = None
for stream in streams:
    if stream['info']['name'][0] == 'Glasses3_Gaze':
        gaze_stream = stream
        break

if gaze_stream is None:
    raise ValueError("Glasses3_Gaze stream not found in XDF file.")

data = gaze_stream['time_series']
timestamps = gaze_stream['time_stamps']

# Gaze data: [local_ts, gaze_ts, pixel_x, pixel_y, norm_x, norm_y]
gaze_x = data[:, 4]
gaze_y = data[:, 5]

# Load video
video_path = 'scenevideo.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1.0 / fps 
cv2.namedWindow('Gaze Overlay', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Gaze Overlay', 960, 540)

# Play video with gaze overlay
timestamps = timestamps - timestamps[0]
gazestartfirst = False
if gazestartfirst:
    offset = 8.0
    valid_indices = timestamps >= offset
    timestamps = timestamps[valid_indices] - offset
    gaze_x = gaze_x[valid_indices]
    gaze_y = gaze_y[valid_indices]

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break 
    
    current_video_time = frame_idx * frame_time
    print(current_video_time)

    # Find the nearest gaze
    gaze_index = np.argmin(np.abs(timestamps - current_video_time))
    gx, gy = gaze_x[gaze_index], gaze_y[gaze_index]
    
    # Convert gaze coordinates to pixel coordinates
    h, w, _ = frame.shape
    px = int(gx * w)
    py = int(gy * h)

    # Draw gaze overlay
    cv2.circle(frame, (px, py), 20, (0, 0, 255), 2)

    # Show frame
    cv2.imshow('Gaze Overlay', frame)
    if cv2.waitKey(int(frame_time * 1000)) & 0xFF == ord('q'):
        break

    frame_idx += 1

cap.release()
cv2.waitKey(500)
cv2.destroyAllWindows()


# Gaze data from Tobii

In [ ]:
import cv2
import json
import numpy as np

# Load gaze data directly from Tobii
file_path = "gazedata"

timestamps = []
gaze2d_x = []
gaze2d_y = []

with open(file_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue  
        try:
            obj = json.loads(line)
            if obj.get("type") == "gaze":
                gaze2d = obj["data"].get("gaze2d", [])
                if gaze2d and len(gaze2d) == 2:
                    timestamps.append(obj["timestamp"])
                    gaze2d_x.append(gaze2d[0])
                    gaze2d_y.append(gaze2d[1])
                else:
                    continue
        except json.JSONDecodeError as e:
            print("Skipping line due to JSON error:", e)

timestamps = np.array(timestamps, dtype=float)
gaze2d_x = np.array(gaze2d_x, dtype=float)
gaze2d_y = np.array(gaze2d_y, dtype=float)

# Load video
video_path = 'scenevideo.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1.0 / fps 
cv2.namedWindow('Gaze Overlay', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Gaze Overlay', 960, 540)

# Play video with gaze overlay
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break 
    
    current_video_time = frame_idx * frame_time

    # Find the nearest gaze
    gaze_index = np.argmin(np.abs(timestamps - current_video_time))
    gx, gy = gaze2d_x[gaze_index], gaze2d_y[gaze_index]
    
    # Convert gaze coordinates to pixel coordinates
    h, w, _ = frame.shape
    px = int(gx * w)
    py = int(gy * h)

    # Draw gaze overlay
    cv2.circle(frame, (px, py), 20, (0, 0, 255), 2)

    # Show frame
    cv2.imshow('Gaze Overlay', frame)
    if cv2.waitKey(int(frame_time * 1000)) & 0xFF == ord('q'):
        break

    frame_idx += 1

cap.release()
cv2.waitKey(500)
cv2.destroyAllWindows()